In [ ]:
# SecurePass Pro — with built-in common-password list & GUI viewer
# Requires: pip install customtkinter
import customtkinter as ctk
import re
import random
import string
import math

# ---------- Appearance ----------
ctk.set_appearance_mode("dark")
ctk.set_default_color_theme("blue")

app = ctk.CTk()
app.title("🔐 SecurePass Pro — Common-password detection")
app.geometry("700x650")

# ---------- Global state ----------
show_password = False
COMMON_PASSWORDS = {
    # Top common / leaked / predictable passwords (case-insensitive)
    "123456","123456789","12345678","12345","qwerty","password","1234567",
    "1234567890","123123","000000","iloveyou","111111","abc123","qwerty123",
    "1q2w3e","admin","qwertyuiop","letmein","welcome","monkey","login","princess",
    "dragon","passw0rd","starwars","football","baseball","master","hello123",
    "hello@123","password1","password123","shadow","sunshine","ashley","donald",
    "password!", "qazwsx","trustno1","654321","superman","azerty","lovely","zaq1zaq1",
    "1qaz2wsx","q1w2e3r4","freedom","whatever","pass@123","test123","1234","0000"
}

# Normalize set for fast case-insensitive checks
COMMON_SET = {p.lower() for p in COMMON_PASSWORDS}

# ---------- Helpers ----------
def calculate_entropy(password):
    charset = 0
    if re.search(r"[a-z]", password): charset += 26
    if re.search(r"[A-Z]", password): charset += 26
    if re.search(r"\d", password): charset += 10
    if re.search(r"[!@#$%^&*(),.?\":{}|<>_\-+=/\\\[\]\{\};:|`~]", password): charset += 32
    entropy = math.log2(charset ** len(password)) if charset and len(password) > 0 else 0
    return round(entropy, 2)

def is_common_password(password):
    """Return True if the password exactly matches a known common password (case-insensitive),
       or is a small variation (common + digits/punct)."""
    p = password.strip().lower()
    if p in COMMON_SET:
        return True
    # check simple variations: remove common separators and trailing numbers
    # e.g., "hello123" -> "hello" (matches), "password!" -> "password"
    stripped = re.sub(r"[^a-z0-9]", "", p)
    # remove trailing digits
    base = re.sub(r"\d+$", "", stripped)
    if base in COMMON_SET and len(base) >= 3:
        return True
    return False

def generate_password(length=14):
    chars = string.ascii_letters + string.digits + "!@#$%^&*()_+-=[]{}|;:,.<>?"
    while True:
        password = ''.join(random.choice(chars) for _ in range(length))
        # avoid generating accidentally common or weak patterns — simple filter:
        if not is_common_password(password):
            break
    password_entry.delete(0, "end")
    password_entry.insert(0, password)
    check_strength_real_time(None)

def toggle_visibility():
    global show_password
    show_password = not show_password
    if show_password:
        password_entry.configure(show="")
        toggle_button.configure(text="🙈 Hide")
    else:
        password_entry.configure(show="*")
        toggle_button.configure(text="👁️ Show")

def get_text_color():
    return "white" if ctk.get_appearance_mode() == "Dark" else "black"

def toggle_theme():
    mode = ctk.get_appearance_mode()
    if mode == "Dark":
        ctk.set_appearance_mode("light")
        theme_button.configure(text="🌙 Dark Mode")
    else:
        ctk.set_appearance_mode("dark")
        theme_button.configure(text="☀️ Light Mode")
    check_strength_real_time(None)

# ---------- Common-list viewer ----------
def open_common_list_window():
    win = ctk.CTkToplevel(app)
    win.title("Common Passwords — Do NOT use these")
    win.geometry("560x420")
    win.transient(app)
    win.grab_set()
    header = ctk.CTkLabel(win, text="Common / Weak Passwords (examples)", font=ctk.CTkFont(size=16, weight="bold"))
    header.pack(pady=(12,6))
    info = ctk.CTkLabel(win, text="These passwords are widely used or predictable. Avoid them even if you add a symbol or digit.", wraplength=520, justify="left")
    info.pack(padx=12, pady=(0,8))
    textbox = ctk.CTkTextbox(win, width=520, height=300)
    textbox.pack(padx=12, pady=(0,12))
    # insert sorted list
    common_sorted = sorted(COMMON_SET)
    textbox.configure(state="normal")
    textbox.delete("0.0", "end")
    for i, p in enumerate(common_sorted, start=1):
        textbox.insert("end", f"{i:02d}. {p}\n")
    textbox.configure(state="disabled")
    close_btn = ctk.CTkButton(win, text="Close", command=win.destroy)
    close_btn.pack(pady=(0,12))

# ---------- Strength check (real-time) ----------
def check_strength_real_time(event):
    password = password_entry.get()
    base_color = get_text_color()

    # reset view flags
    view_common_button.pack_forget()

    if len(password) == 0:
        result_label.configure(text="Enter a password...", text_color="yellow")
        suggestion_label.configure(text="")
        strength_bar.set(0)
        entropy_label.configure(text="")
        breach_label.configure(text="")
        return

    # basic checks
    strength = 0
    suggestions = []
    if len(password) >= 8:
        strength += 1
    else:
        suggestions.append("Use at least 8 characters (8+).")
    if re.search(r"[A-Z]", password):
        strength += 1
    else:
        suggestions.append("Add uppercase letters (A-Z).")
    if re.search(r"[a-z]", password):
        strength += 1
    else:
        suggestions.append("Add lowercase letters (a-z).")
    if re.search(r"\d", password):
        strength += 1
    else:
        suggestions.append("Add numbers (0-9).")
    if re.search(r"[!@#$%^&*(),.?\":{}|<>_\-+=/\\\[\]\{\};:|`~]", password):
        strength += 1
    else:
        suggestions.append("Add special characters (e.g. @, #, $).")

    # detect common passwords and downgrade / warn
    common_flag = is_common_password(password)
    if common_flag:
        # add a specific suggestion/warning
        suggestions.append("⚠️ This password is too common or predictable — do NOT use it.")
        # ensure it appears weak to the user
        strength = min(strength, 2)

    # update visual strength
    strength_bar.set(strength / 5)
    entropy = calculate_entropy(password)
    entropy_label.configure(text=f"🔢 Entropy: {entropy} bits", text_color=base_color)

    if strength <= 2:
        result_label.configure(text="🔴 Weak", text_color="red")
    elif strength == 3 or strength == 4:
        result_label.configure(text="🟡 Moderate", text_color="orange")
    else:
        result_label.configure(text="🟢 Strong", text_color="green")

    # suggestions text
    if suggestions:
        suggestion_text = "\n".join(f"• {s}" for s in suggestions)
        suggestion_label.configure(text=f"Suggestions:\n{suggestion_text}", text_color=base_color)
    else:
        suggestion_label.configure(text="✅ Your password looks strong!", text_color="green" if ctk.get_appearance_mode() == "Light" else "lightgreen")

    # breach_label: local simulation
    if len(password) < 6:
        breach_label.configure(text="⚠️ Too short — higher breach risk.", text_color="orange")
    else:
        breach_label.configure(text="✅ No known breaches found (local check).", text_color="blue" if ctk.get_appearance_mode()=="Light" else "lightblue")

    # if password is common, show 'View common passwords' button
    if common_flag:
        view_common_button.pack(pady=(6,0))

# ---------- UI Layout ----------
title = ctk.CTkLabel(app, text="🔐 SecurePass Pro", font=ctk.CTkFont(size=26, weight="bold"))
title.pack(pady=(18,6))

subtitle = ctk.CTkLabel(app, text="Real-time strength · entropy · generator · common-password detection", font=ctk.CTkFont(size=11))
subtitle.pack(pady=(0,12))

# entry
password_entry = ctk.CTkEntry(app, placeholder_text="Enter or generate password", width=480, show="*")
password_entry.pack(pady=(6,8))
password_entry.bind("<KeyRelease>", check_strength_real_time)

# controls row
controls_frame = ctk.CTkFrame(app)
controls_frame.pack(pady=(6,12), padx=12)

toggle_button = ctk.CTkButton(controls_frame, text="👁️ Show", width=110, command=toggle_visibility)
toggle_button.grid(row=0, column=0, padx=8, pady=8)

generate_button = ctk.CTkButton(controls_frame, text="🔐 Generate", width=110, command=lambda: generate_password(14))
generate_button.grid(row=0, column=1, padx=8, pady=8)

theme_button = ctk.CTkButton(controls_frame, text="☀️ Light Mode", width=140, command=toggle_theme)
theme_button.grid(row=0, column=2, padx=8, pady=8)

# result + strength bar
result_label = ctk.CTkLabel(app, text="", font=ctk.CTkFont(size=18))
result_label.pack(pady=(14,6))

strength_bar = ctk.CTkProgressBar(app, width=520)
strength_bar.set(0)
strength_bar.pack(pady=6)

entropy_label = ctk.CTkLabel(app, text="", font=ctk.CTkFont(size=13))
entropy_label.pack(pady=(6,2))

suggestion_label = ctk.CTkLabel(app, text="", justify="left", wraplength=640, font=ctk.CTkFont(size=13))
suggestion_label.pack(pady=(8,6))

breach_label = ctk.CTkLabel(app, text="", font=ctk.CTkFont(size=12))
breach_label.pack(pady=(2,4))

# view common list button (hidden until needed)
view_common_button = ctk.CTkButton(app, text="View common passwords", command=open_common_list_window)

# footer / quick examples
examples = ctk.CTkLabel(app, text="Examples to test: hello@123, 123456, P@ssw0rd! (try them)", font=ctk.CTkFont(size=11))
examples.pack(pady=(18,8))

# run
check_strength_real_time(None)  # initialize UI text/colors
app.mainloop()


In [ ]:

content = """🔐 SecurePass Pro — Intelligent Password Strength Checker (Jupyter Edition)

SecurePass Pro is a modern GUI-based password strength analyzer built entirely in Python using CustomTkinter — right inside a Jupyter Notebook.
It helps users instantly test and improve password strength, provides entropy calculations, detects common passwords, and offers smart suggestions — all in a visually interactive interface.

────────────────────────────────────────────
🌟 Key Features
────────────────────────────────────────────
✅ Real-time Strength Indicator
Instantly updates as you type — showing strength as Weak, Moderate, or Strong.

✅ Entropy / Score Meter
Displays entropy bits that reflect password randomness and complexity.

✅ Common Password Detection
Contains 50+ common and predictable passwords (like 123456, password, hello@123) and warns users if their password is too common.
A “View Common Passwords” button lets users see the full list of weak examples.

✅ Password Generator 🔐
Generates strong, random passwords avoiding common patterns.

✅ Password Visibility Toggle 👁️
Allows you to show or hide your entered password safely.

✅ Dark / Light Theme 🌙☀️
Switch themes dynamically with adaptive colors for perfect visibility.

✅ Offline Mode
Runs entirely offline — no internet or external API required.

────────────────────────────────────────────
🧠 How It Works
────────────────────────────────────────────
• The app uses entropy-based analysis, character variety, and length to score passwords.
• Detects common passwords or variations (e.g. “hello@123”).
• Gives human-friendly suggestions for improvement.
• Includes a random secure password generator.
• GUI automatically updates with every keystroke.

────────────────────────────────────────────
🧩 Project Structure
────────────────────────────────────────────
SecurePassPro/
│
├── securepass.ipynb        # Main Jupyter Notebook (app interface)
├── README.txt              # Project documentation (this file)
├── dark_mode.png           # Optional screenshot
├── light_mode.png          # Optional screenshot
└── requirements.txt        # Optional dependencies file

────────────────────────────────────────────
⚙️ Setup & Installation
────────────────────────────────────────────
Step 1️⃣ — Install Required Packages
In a new Jupyter Notebook cell, run:
!pip install customtkinter

Step 2️⃣ — Open the Notebook
Open securepass.ipynb in Jupyter Notebook or VS Code (with Jupyter support).

Step 3️⃣ — Run the App
Run the cell that contains the SecurePass Pro code.
The GUI window will appear — you can:
• Type or paste a password to check its strength
• Generate random passwords 🔐
• Toggle visibility 👁️
• Switch dark/light themes 🌗
• View common passwords ⚠️

────────────────────────────────────────────
🧩 Example Password Tests
────────────────────────────────────────────
Password      | Strength | Comment
--------------|-----------|----------------------------------
123456        | 🔴 Weak   | Extremely common
Hello@123     | 🔴 Weak   | Common pattern
Pa$$w0rd!     | 🟡 Moderate | Variation of “password”
G7!rM2z#pL9q  | 🟢 Strong | Excellent entropy

────────────────────────────────────────────
🧰 Technologies Used
────────────────────────────────────────────
• Python 3.11.9
• CustomTkinter (modern GUI framework)
• Regex (pattern matching)
• Math (entropy calculation)
• String (password generation)

────────────────────────────────────────────
💡 Future Enhancements
────────────────────────────────────────────
• ✅ Larger common password database (10K+ entries)
• ✅ Fuzzy detection for passwords containing common words
• ⏳ Integration with HaveIBeenPwned API for breach checks
• ⏳ Export strong passwords to a text file
• ⏳ More UI customization and animations

────────────────────────────────────────────
🧑‍💻 Author
────────────────────────────────────────────
Zain Ul Abdin  
💻 Cybersecurity Enthusiast | Ethical Hacker in Training | Python Developer  
📍 Pakistan  
GitHub: https://github.com/your-username  
LinkedIn: https://linkedin.com/in/your-link  

────────────────────────────────────────────
🛡️ License
────────────────────────────────────────────
This project is released under the MIT License — you are free to use, modify, and distribute it.

────────────────────────────────────────────
⚠️ Disclaimer
────────────────────────────────────────────
This project is for learning and cybersecurity awareness purposes only.
It performs all checks locally and never sends your passwords over the internet.
Always use unique, complex passwords for each online account.

⭐ If you find SecurePass Pro helpful, please star the repository!
"""

# Save to file
with open("README.txt", "w", encoding="utf-8") as file:
    file.write(content)

print("✅ README.txt has been successfully created in your project folder!")


✅ README.txt has been successfully created in your project folder!
